In [1]:
from pyspainmobility import Mobility, Zones
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

/Users/mluca/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:
# getting 1 week of mobility data. In this case, we download the data from March 10 to March 16 
mobility_data = Mobility(version=2, zones='lua', start_date='2024-10-14', end_date='2024-10-20', social_agg=True)
# and we extract the OD matrices 
mobility_data.get_od_data()

In [ ]:
df = pd.read_parquet('Viajes_GAU_2024-10-14_2024-10-20_v2.parquet')
df = df[(df['destino'].str.contains('GAU Barcelona'))|(df['origen'].str.contains('GAU Barcelona'))]

In [ ]:
# Prepare and clean data
df_age_period = df[['age', 'date', 'n_trips']].dropna()
df_age_period['date'] = df_age_period['date'].astype(int)

# Filter only valid age groups
valid_ages = ['0-25', '25-45', '45-65', '65-100']
df_age_period = df_age_period[df_age_period['age'].isin(valid_ages)]

# Aggregate data
grouped = df_age_period.groupby(['date', 'age'])['n_trips'].sum().reset_index()

# Set Seaborn style
sns.set(style="whitegrid")

# Custom color palette
custom_colors = ["#FFA500", "#FF8C00", "#FF3344", "#FF66CC"]
palette = dict(zip(valid_ages, custom_colors))

# Plot
plt.figure(figsize=(12, 6))
ax = sns.lineplot(data=grouped, x='date', y='n_trips', hue='age', palette=palette)

# Remove top and right borders
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Emphasize bottom and left borders
ax.spines['bottom'].set_linewidth(1.5)
ax.spines['left'].set_linewidth(1.5)
ax.spines['bottom'].set_color('black')
ax.spines['left'].set_color('black')

# Labels and layout
plt.title("Hourly Trip Volume by Age Group", fontsize=18)
plt.xlabel("Hour of Day")
plt.ylabel("Number of Trips")
plt.legend(title="Age Group")
plt.tight_layout()
plt.show()
